In [2]:
import mlflow
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import pandas as pd
import matplotlib.pyplot as plt
from category_encoders import TargetEncoder
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [3]:
# we want to keep track of our experiments - Every project is an experiment
# within the experiment we can have multiple runs/ models
# set the name of our experiment
mlflow.set_experiment("loan_default_pred")

2026/01/24 16:18:06 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/01/24 16:18:06 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/01/24 16:18:06 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/01/24 16:18:06 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/01/24 16:18:06 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/01/24 16:18:06 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/01/24 16:18:15 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/24 16:18:16 INFO mlflow.store.db.utils: Updating database tables
2026/01/24 16:18:16 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/24 16:18:16 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/24 16:18:17 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/01/24 16:1

<Experiment: artifact_location='file:c:/Users/itviv/Documents/GitHub/learning_streamlit/mlruns/1', creation_time=1769251704081, experiment_id='1', last_update_time=1769251704081, lifecycle_stage='active', name='loan_default_pred', tags={}>

In [ ]:
mlflow.enable_system_metrics_logging()

In [4]:
data = pd.read_csv("LoanTap_Data_Streamlit.csv")

C:\Users\itviv\AppData\Local\Temp\ipykernel_15000\2723465646.py:1: DtypeWarning: Columns (2,4,5,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("LoanTap_Data_Streamlit.csv")


In [ ]:
data = data.dropna()

loan_label= {'Fully Paid':0, 'Charged Off':1}
data['loan_status'] = data['loan_status'].replace(loan_label)
data['zip_code'] = data['address'].astype(str).str[-5:]

X = data[data.columns.drop(['loan_status','address'])]
y = data['loan_status']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

cat_cols = ['grade','zip_code','application_type','purpose']

te = TargetEncoder(cols=cat_cols)
X_train[cat_cols] = te.fit_transform(X_train[cat_cols], y_train)
X_test[cat_cols] = te.transform(X_test[cat_cols])

C:\Users\itviv\AppData\Local\Temp\ipykernel_15000\1780818132.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['loan_status'] = data['loan_status'].replace(loan_label)


In [ ]:
mlflow.enable_system_metrics_logging()

# Anything that u want to track will come under the below piece of code "mlflow.start_run"
# Experiment starts running from here
with mlflow.start_run():

    n_estimators = 1000
    learning_rate = 0.06
    max_depth = 20
    xgb_model = XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth
    )

    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)

    mape = mean_absolute_percentage_error(y_test, y_pred)

    # Storing the hyperparameters, model metrics etc
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_metric("mape", mape)

    # Type python -m mlflow ui